In [5]:
import pandas as pd
import numpy as np
from glob import glob
import time
from datetime import datetime
import csv
import ccxt

from data_provider import read_data_path, market_to_symbol

market = 'ONT-BTC'
tf = '15m'

state_fn = 'Logs/' + robot.robot_name + '.state'
state_file = glob(state_fn)
if state_file == []:
    market_position = 0
    start_capital = 0.0057
    amount = 0
else:
    print('State file !!!')
    df_state = pd.read_csv(state_file[-1])    
    market_position = df_state['mp'][0]
    start_capital = df_state['capital'][0]
    amount = df_state['amount'][0]

deposit_percent = 0.3
commision = 0.001

#Work 30
api_key = "mZrSEDmM7GJ7dCBT5oKcTa5pKdV8CXeEQoR4nlLGPTBWtn9GWqi0UFXLyFG8E6Ry"
api_secret = "bLtJGjMH0gRRtvDa7OXifGlgpFM2NrsbmOXBnt8tyUuigShZwm2MissRO2PeAKeT"

pd.set_option('precision', 9)
exchange = ccxt.binance({'apiKey': api_key,
                         'secret': api_secret})

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid

from parfit import bestFit # Necessary if you wish to use bestFit
from parfit.score import roc_auc_score

def train_clf():
grid = {
    'min_samples_leaf': [1, 5, 10, 15, 20, 25],
    'max_features': ['sqrt', 'log2', 0.5, 0.6, 0.7],
    'n_estimators': [60],
    'random_state': [42],
    'n_jobs':[-1]
}
paramGrid = ParameterGrid(grid)

# Necessary if you wish to run each step sequentially
#from parfit.fit import *
#from parfit.plot import *
#from parfit.crossval import *

start_time = time.time()
lags = 30
res = []
for market in markets:
    for kline in timeframes:
        prices = read_prices(market, kline)
        returns = np.diff(np.log(prices))
        returns = returns[returns!=0]
        data = []
        for k in range(lags):
            xx = np.roll(returns, k)
            data.append(xx)

        data = np.array(data)
        data = data.transpose()
        data = data[lags-1:]
        data_binary = np.sign(data)

        train = int(0.7 * data.shape[0])

        y_train = data_binary[:train, 0]
        X_train = data[:train, 1:]
        y_val   = data_binary[train:, 0]
        X_val   = data[train:, 1:]


        best_model, best_score, all_models, all_scores = bestFit(RandomForestClassifier(), paramGrid,
                                                    X_train, y_train, X_val, y_val, nfolds=5, #[optional, instead of validation set]
                                                    metric=roc_auc_score, greater_is_better=True, 
                                                    scoreLabel='AUC',
                                                    showPlot = False)
        print(market, kline, round(best_score,3))
        res.append([market, kline, best_score])
        df = pd.DataFrame(res)
        df.to_csv('Logs/res_pr-sklearn_' + market + '_' + kline + '.csv')
print("Time taken = {0:.5f}".format(time.time() - start_time))

In [6]:
def write_trades(df_trades, trade):
    df_state = pd.DataFrame(columns=['mp', 'capital', 'amount'])
    df_state.loc[1] = [market_position, cur_capital, amount]
    df_state.to_csv(state_fn, index = False)
    
    print(trade[1])
    print('{d[0]} {d[1]:10s} {d[2]:.2f} {d[3]:.8f} {d[4]: .8f} {d[5]:.8f} {d[6]:.8f} {d[7]:.8f}'.format(d=trade))
    df_trades.loc[len(df_trades)] = trade
    fn =  'Logs/trades_' + robot.robot_name + '.csv'
    df_trades.to_csv(fn , index = False)    
    

def create_market_buy_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_buy_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price

def create_market_sell_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_sell_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price
  

In [ ]:
data_path = read_data_path()
df_trades = pd.DataFrame(columns = ['Date', 'Trade type', 'amount', 'close price', 'buy price', 'cumProfit', 'curCapital', 'Deposit'])

symbol = market_to_symbol(market)
cur_capital = start_capital
cum_profit  = 0
buy_sum     = 0
deposit = 0
last_time = ''
tf_since_entry = 0

stop= False
print('Start trading:', str(datetime.now()), 'cur_capital:', cur_capital, 'amount:', amount)
while not stop:
    time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
    #pd.read_csv не работает с сash file
    with open(data_path + '/Cash/' + market + '_' + tf + '.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        rl = list(reader)
    if rl != []:
        df = pd.DataFrame(rl[1:])
        df.columns = rl[0]
    if last_time > df.iloc[-1]['T']:
        print('time error', last_time, df.iloc[-1]['T'])
        continue
    elif last_time < df.iloc[-1]['T']:
        last_time = df.iloc[-1]['T']
        tf_since_entry += 1           
    
            
    tf_closes = df["C"].values.astype('float')
    
    price = tf_closes[-1]
    close_time = exchange.iso8601(int(df.iloc[-1]['T'])) 
    signals = robot.signals(tf_closes, *robot_parameters)
    
    #Long entry 
    if  market_position == 0 and signals[-1] == 1:

        amount = cur_capital / price * (1-commision) 
        amount = np.trunc(amount * 100)/100
        buy_price = create_market_buy_order(exchange, symbol, amount)
        
        buy_sum = buy_price * amount + cur_capital * commision
        cur_capital = 0;

        market_position = 1
        tf_since_entry = 0    
 
     
        write_trades(df_trades, [close_time, "entry", amount, price,  buy_price, cum_profit, cur_capital, deposit])
      #Long Exit Profit 
    if market_position == 1 and tf_since_entry > 1 and signals[-1] == -1:
 
        sell_price = create_market_sell_order(exchange, symbol, amount)

        profit = sell_price * amount - buy_sum
        cum_profit += profit
        deposit_sum = max(0, profit) * deposit_percent
        deposit += deposit_sum
        cur_capital = sell_price * amount - deposit_sum
        buy_sum = 0

        market_position = 0

        write_trades(df_trades, [close_time, "exit", amount, price, sell_price , cum_profit, cur_capital, deposit])
    #------------------Post processing---------------------------------------------------


    with open(state_fn, 'w') as state_file:
        state_file.write("time,mp,capital,amount\n") 
        state_file.write("%s,%s,%s,%s, %s" % (close_time, market_position, cur_capital, amount, buy_sum))

    df_stop = pd.read_csv('markets_in_trade.csv')
    stop=df_stop[(df_stop['robot'] == robot.robot_name) & (df_stop['market'] == market) & (df_stop['tf'] == tf)]['stop']
    if stop.shape[0] == 0: 
        print('В файле markets_in_trade.csv нет установок для данного робота')
        break
    else:
        stop = bool(stop.values[0])
    if stop:
        if market_position == 1: 
            sell_price = create_market_sell_order(exchange, symbol, amount)

            profit = sell_price * amount - buy_sum
            cum_profit += profit
            deposit_sum = max(0, profit) * deposit_percent
            deposit += deposit_sum
            cur_capital = sell_price * amount - deposit_sum

            market_position = 0

            write_trades(df_trades, [close_time, "stop", amount, price, sell_price , cum_profit, cur_capital, deposit])
        break

Start trading: 2018-05-06 23:11:14.603908 cur_capital: 0.0057 amount: 0
entry
2018-05-07T01:30:00.000Z entry      6.22 0.00091420  0.00091470 0.00000000 0.00000000 0.00000000
exit
2018-05-07T17:15:00.000Z exit       6.22 0.00103170  0.00102860 0.00070276 0.00618706 0.00021083
entry
2018-05-08T03:15:00.000Z entry      6.25 0.00098850  0.00098850 0.00070276 0.00000000 0.00021083
